<h1><center> RECOMMENDATION SYSTEM

## Importing necessary libraries

In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow_text as text
import time
import sklearn.metrics as metrics
import re

## Reading the dataset

In [2]:
youtube = pd.read_csv('GB_videos_data.csv')
youtube = youtube.drop_duplicates(subset = ['title'])
youtube = youtube[['title', 'description']]
youtube.columns = ['Title', 'Description']
youtube['cleaned_title'] = youtube['Title'].apply(lambda x: x.lower())
youtube['cleaned_title'] = youtube['cleaned_title'].apply(lambda x: re.sub('[^A-Za-z0-9]+', ' ', x))

In [3]:
youtube

,Title,Description,cleaned_title
0,Milan 0-3 Inter | Inter Go Four Points Clear w...,Inter breezed past Milan in the derby to go fo...,milan 0 3 inter inter go four points clear wit...
1,PE With Joe 2021 | Monday 22nd Feb,,pe with joe 2021 monday 22nd feb
2,Cards Against Humanity PHAN EDITION!,We play an offensive card game ..created by Da...,cards against humanity phan edition
3,HIGHLIGHTS! David Avanesyan crushes Josh Kelly...,►SUBSCRIBE http://bit.ly/SSBoxingSubJosh Kelly...,highlights david avanesyan crushes josh kelly ...
4,WHO IS THE BEST COUPLE?!,"This was so crazy, you will never guess who ma...",who is the best couple
...,...,...,...
5869,NIGELLA LAWSON So Pretty But CAN SHE MAKE RAMEN?,"Niece and nephew, go watch Nephew Nigel podcas...",nigella lawson so pretty but can she make ramen
5872,YouTube just isn't working for me anymore,I think I've reached my breaking point with Yo...,youtube just isn t working for me anymore
5880,Solskjaer's Last Chance! Pogba Juve Talks? Man...,Man Utd must change under Solskjaer if United ...,solskjaer s last chance pogba juve talks man u...
5887,Bloodcurdling Facts That You Wish You Never Kn...,► TikTok: https://www.tiktok.com/@fyp► Instagr...,bloodcurdling facts that you wish you never kn...


In [4]:
preprocessor = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1", trainable=True)

## 1) BERT on Title Column

In [5]:


def get_bert_embeddings(text, preprocessor, encoder):
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
    encoder_inputs = preprocessor(text_input)
    outputs = encoder(encoder_inputs)
    embedding_model = tf.keras.Model(text_input, outputs['pooled_output'])
    sentences = tf.constant([text])
    return embedding_model(sentences)



In [6]:
start_time = time.time()
youtube['encodings'] = youtube['cleaned_title'].apply(lambda x: get_bert_embeddings(x, preprocessor, encoder))
print("--- %s seconds ---" % (time.time() - start_time))

--- 86.87068486213684 seconds ---


In [7]:
youtube['encodings'][0]

<tf.Tensor: shape=(1, 128), dtype=float32, numpy=
array([[-0.9996609 ,  0.00611476, -0.90465075,  0.50488615, -0.9914023 ,
         0.8864676 , -0.9676053 , -0.5649521 ,  0.00441299,  0.11323541,
        -0.66438246, -0.01549171,  0.03428588,  0.9784331 , -0.3433421 ,
         0.7791186 ,  0.993968  ,  0.04071885, -0.9056964 , -0.4641791 ,
         0.91723204, -0.00427914, -0.85351753, -0.92165285, -0.99904186,
         0.03395874, -0.9945298 ,  0.99323004,  0.852847  ,  0.09699281,
        -0.00655349, -0.19616483,  0.01363273, -0.91710025,  0.91037726,
         0.9801209 , -0.9887686 , -0.00849838, -0.889948  , -0.8646063 ,
         0.9871473 ,  0.8467633 , -0.89535064,  0.866157  , -0.73643064,
        -0.07725015, -0.4894271 ,  0.9762503 , -0.05012548,  0.9958873 ,
         0.57535356, -0.9566275 ,  0.05867821,  0.9872388 ,  0.982109  ,
         0.52845955, -0.3528041 , -0.9294477 ,  0.9626395 , -0.7742015 ,
         0.05273832,  0.97950155,  0.8175247 ,  0.52698344, -0.4604506 ,
 

In [33]:
def preprocess_text():
    text = input()
    text = text.lower()
    text = re.sub('[^A-Za-z0-9]+', ' ', text)
    return text
  
query_text = preprocess_text()
query_encoding = get_bert_embeddings(query_text, preprocessor, encoder)

youtube['similarity_score'] = youtube['encodings'].apply(lambda x: metrics.pairwise.cosine_similarity(x, query_encoding)[0][0])
df_results = youtube.sort_values(by=['similarity_score'], ascending=False)

Football


KeyError: 'encodings'

In [9]:
query_text

'music'

In [34]:
df_results

,Title,Description,cleaned_title,encodings,similarity_score
5649,Quadeca - Sisyphus (Official Music Video),Pre-save to Spotify/Apple Music here! https://...,quadeca sisyphus official music video,"((tf.Tensor(-0.9999955, shape=(), dtype=float3...",0.974822
422,선미(SUNMI) - 꼬리(TAIL) MUSIC VIDEO,[MV] 선미(SUNMI) - 꼬리(TAIL) 뮤직비디오⠀💿 선미(SUNMI) - ...,sunmi tail music video,"((tf.Tensor(-0.9999925, shape=(), dtype=float3...",0.963242
2043,Sub Urban - Cirque [Official Music Video],MERCH & ANNOUNCEMENTS: https://www.thatsuburba...,sub urban cirque official music video,"((tf.Tensor(-0.99998575, shape=(), dtype=float...",0.961027
5478,Pop Smoke - AP (Official Music Video),"Official music video for Pop Smoke AP, availab...",pop smoke ap official music video,"((tf.Tensor(-0.999985, shape=(), dtype=float32...",0.955097
3627,Giveon - Heartbreak Anniversary (Official Musi...,When It's All Said And Done...Take Time availa...,giveon heartbreak anniversary official music v...,"((tf.Tensor(-0.9999778, shape=(), dtype=float3...",0.954379
...,...,...,...,...,...
467,D-low | Round 2 - Quarterfinal 3 | RYTHMIND vs...,Welcome to round 2 of the third quarter final ...,d low round 2 quarterfinal 3 rythmind vs d low...,"((tf.Tensor(-0.9999254, shape=(), dtype=float3...",0.635841
401,Atletico Madrid v Chelsea (0-1) | Giroud score...,Olivier Giroud's magnificent bicycle-kick was ...,atletico madrid v chelsea 0 1 giroud scores am...,"((tf.Tensor(-0.99856704, shape=(), dtype=float...",0.591604
4116,Knights v Bulldogs Match Highlights | Round 1 ...,Knights co-captain Daniel Saifiti celebrated h...,knights v bulldogs match highlights round 1 20...,"((tf.Tensor(-0.99970746, shape=(), dtype=float...",0.584821
5013,UEFA Champions League 2020/21 Quarter-Final & ...,Watch the draw for 2020/21 UEFA Champions Leag...,uefa champions league 2020 21 quarter final se...,"((tf.Tensor(-0.99868125, shape=(), dtype=float...",0.559690


In [35]:
results=df_results[['Title','Description','similarity_score']].copy()

In [36]:
final_results=results.head(10)
final_results

,Title,Description,similarity_score
5649,Quadeca - Sisyphus (Official Music Video),Pre-save to Spotify/Apple Music here! https://...,0.974822
422,선미(SUNMI) - 꼬리(TAIL) MUSIC VIDEO,[MV] 선미(SUNMI) - 꼬리(TAIL) 뮤직비디오⠀💿 선미(SUNMI) - ...,0.963242
2043,Sub Urban - Cirque [Official Music Video],MERCH & ANNOUNCEMENTS: https://www.thatsuburba...,0.961027
5478,Pop Smoke - AP (Official Music Video),"Official music video for Pop Smoke AP, availab...",0.955097
3627,Giveon - Heartbreak Anniversary (Official Musi...,When It's All Said And Done...Take Time availa...,0.954379
126,Ariana Grande - main thing (official audio),main thing” from Positions (Deluxe). Listen & ...,0.954103
3602,Central Cee - 6 For 6 [Music Video],📀 Wild West Mixtape: https://ada.lnk.to/Wild-W...,0.953150
3633,SION - The Blade (Official Music Video),Howard Jones - VocalsJared Dines - GuitarHiram...,0.952570
3043,What Makes This Song Great? Ep.100 ?,In this episode of What Makes This Song Great?...,0.950409
4451,펜타곤(PENTAGON) - 'DO or NOT' Official Music Video,펜타곤(PENTAGON) - 'DO or NOT' Official Music Vid...,0.948621


##### Inference: BERT is working well with title column. The results returned are according to the given input tag "Music"

## 2) SBERT on  Title column

In [13]:
youtube = pd.read_csv('GB_videos_data.csv')
youtube = youtube.drop_duplicates(subset = ['title'])
youtube = youtube[['title', 'description']]
youtube.columns = ['Title', 'Description']
youtube['Title'] = youtube['Title'].apply(lambda x: x.lower())

In [14]:
# youtube['index']=[i for i in range(0,len(youtube))]

In [15]:
youtube

,Title,Description
0,milan 0-3 inter | inter go four points clear w...,Inter breezed past Milan in the derby to go fo...
1,pe with joe 2021 | monday 22nd feb,
2,cards against humanity phan edition!,We play an offensive card game ..created by Da...
3,highlights! david avanesyan crushes josh kelly...,►SUBSCRIBE http://bit.ly/SSBoxingSubJosh Kelly...
4,who is the best couple?!,"This was so crazy, you will never guess who ma..."
...,...,...
5869,nigella lawson so pretty but can she make ramen?,"Niece and nephew, go watch Nephew Nigel podcas..."
5872,youtube just isn't working for me anymore,I think I've reached my breaking point with Yo...
5880,solskjaer's last chance! pogba juve talks? man...,Man Utd must change under Solskjaer if United ...
5887,bloodcurdling facts that you wish you never kn...,► TikTok: https://www.tiktok.com/@fyp► Instagr...


In [16]:
youtube=youtube.reset_index(drop=True)

In [17]:
youtube

,Title,Description
0,milan 0-3 inter | inter go four points clear w...,Inter breezed past Milan in the derby to go fo...
1,pe with joe 2021 | monday 22nd feb,
2,cards against humanity phan edition!,We play an offensive card game ..created by Da...
3,highlights! david avanesyan crushes josh kelly...,►SUBSCRIBE http://bit.ly/SSBoxingSubJosh Kelly...
4,who is the best couple?!,"This was so crazy, you will never guess who ma..."
...,...,...
1365,nigella lawson so pretty but can she make ramen?,"Niece and nephew, go watch Nephew Nigel podcas..."
1366,youtube just isn't working for me anymore,I think I've reached my breaking point with Yo...
1367,solskjaer's last chance! pogba juve talks? man...,Man Utd must change under Solskjaer if United ...
1368,bloodcurdling facts that you wish you never kn...,► TikTok: https://www.tiktok.com/@fyp► Instagr...


In [18]:
# def title(index):
#     return youtube[youtube.index==index]['Title'].values[0]

# def index(Title):
#     return youtube[youtube.Title==Title]['index'].values[0]

In [37]:
from sentence_transformers import SentenceTransformer

In [38]:
# youtube[youtube.Title=="Football"]['index']

In [39]:
sbert=SentenceTransformer('all-MiniLM-L6-v2')

In [40]:
start_time = time.time()
sentence_emb=sbert.encode(youtube['Title'].tolist())
print("--- %s seconds ---" % (time.time() - start_time))


--- 14.160491466522217 seconds ---


##### Inference: It only took 14 econds for SBERT to run on title column

In [41]:
sentence_emb

array([[-0.0906276 ,  0.09738562, -0.00137586, ...,  0.01960378,
        -0.06531686,  0.01742022],
       [-0.05565025,  0.03366059,  0.00368718, ..., -0.06715932,
        -0.04094475, -0.03570421],
       [-0.09258084,  0.1110369 , -0.04113927, ..., -0.03599366,
        -0.05138804,  0.0741779 ],
       ...,
       [-0.05239312,  0.03886008,  0.03703387, ..., -0.01047035,
        -0.04757101,  0.04726708],
       [-0.00348634,  0.1059471 ,  0.00174361, ...,  0.01416963,
        -0.04224442, -0.04140633],
       [-0.00710211,  0.04435631, -0.03698041, ...,  0.0213952 ,
         0.08395005,  0.04220301]], dtype=float32)

In [42]:
np.shape(sentence_emb)

(1370, 384)

In [43]:
inp_val=input('Enter your tag here')
inp_val=inp_val.lower()
inp=[]
inp.append(inp_val)

Enter your tag hereFootball


In [44]:
inp_emb=sbert.encode(inp)

In [45]:
youtube['SbertEmb']=''

In [46]:
for i in range(0,np.shape(sentence_emb)[0]):
    youtube['SbertEmb'][i]=sentence_emb[i]

<ipython-input-46-6c894953625d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  youtube['SbertEmb'][i]=sentence_emb[i]


In [47]:
# youtube

In [48]:
youtube['similarity_score']=youtube['SbertEmb'].apply(lambda x:metrics.pairwise.cosine_similarity(np.array(x).reshape(1,-1).tolist(),inp_emb)[0][0])

In [49]:
df_results_SBert=youtube.sort_values(by=['similarity_score'], ascending=False)

In [50]:
df_results_SBert.head(10)

,Title,Description,SbertEmb,similarity_score
1178,"criminal fossil football, a disgrace, out of e...",#COYS \rSUBSCRIBE BELOW\r\rINSTAGRAM\rhttps://...,"[-0.030581718, 0.09058389, 0.017611284, -0.102...",0.412248
311,"state of play | february 25, 2021 [english]","Rated RP - T, THIS LIVE STREAM MAY INCLUDE CON...","[-0.019910697, -0.015395581, 0.041973628, -0.0...",0.407070
1163,man city are ruining football! goldbridge reacts,Mark Goldbridge says Man City are ruining foot...,"[0.035649724, 0.027067587, 0.067696184, -0.015...",0.394250
689,bayern munich vs borussia dortmund (4-2) | lew...,Robert Lewandowski's hat-trick cancels out an ...,"[-0.03715913, 0.16277362, -0.059222616, -0.038...",0.353348
1042,england v france (23-20) | guinness six nation...,Subscribe to ITV Sport: https://www.youtube.co...,"[-0.0077256616, 0.06280544, -0.033587497, -0.0...",0.350763
107,real sociedad vs. manchester united (0-4) | br...,Man Utd all but secured passage to the next ro...,"[-0.029222785, 0.04029777, -0.0887379, -0.0575...",0.348186
1340,ball magic #shorts,my first #short 👅https://www.curiositybox.com,"[0.024592932, 0.10318325, 0.011696474, -0.0606...",0.344452
1275,leicester city v manchester united | key momen...,Leicester City hosted Manchester United in the...,"[0.00399212, 0.07509963, -0.07613443, -0.01144...",0.341071
1349,video games in real life,"From Rust to Among Us, here’s what your favori...","[-0.042335514, -0.021696538, -0.03548399, -0.0...",0.337534
975,pregnant!,Pregnant! All of our videos are closed caption...,"[-0.06418139, 0.0065174005, -0.00035676156, 0....",0.335168


##### Inference: Even with lower similarity scores, SBERT shows pretty accurate results

### continued in second report.......